In [0]:
!pip install gym-super-mario-bros
!pip install import-ipynb
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive

In [0]:
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
Drive = GoogleDrive(gauth)
#importando wrapper
# https://drive.google.com/open?id=1hk5V6dPPgcAFNCqTlDwQRM_dlqdxCEwA
my_wrapper = Drive.CreateFile({'id':'1hk5V6dPPgcAFNCqTlDwQRM_dlqdxCEwA'})
my_wrapper.GetContentFile('env_wrapper.ipynb')
import env_wrapper as wrappers
#importando modelo de rede dqn
#https://drive.google.com/open?id=14wjjrd_iI1y1x-N01PJaDBzqC43c1ipd
dqn_model = Drive.CreateFile({'id':'14wjjrd_iI1y1x-N01PJaDBzqC43c1ipd'})
dqn_model.GetContentFile('dqn.ipynb')
import dqn as dqn_model

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#!ls "/content/drive/My Drive/DQN"
%cd "/content/drive/My Drive/DQN/Logs"

In [5]:
!pwd

/content/drive/My Drive/DQN/Logs


In [0]:
import time
import datetime
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import csv

In [0]:
!nvidia-smi

In [0]:
DEFAULT_ENV_NAME = 'SuperMarioBros-1-1-v0'

# Valor de Gamma usado na aproximação de Bellman
GAMMA = 0.99
# O tamanho da 'batch' amostrada do Replay Buffer
BATCH_SIZE = 32
# Capacidade máxima do Replay Buffer
REPLAY_SIZE = 10000
# Número de frames que esperamos antes de começar a treinar para popular o Replay Buffer
REPLAY_START_SIZE = 10000
# Taxa de aprendizado usada pelo otimizador Adam, que é usado nesse código
LEARNING_RATE = 1e-4
# Frequência de sincronização dos pesos do modelo de treino para o modelo alvo.
SYNC_TARGET_FRAMES = 1000

""" Parâmetros de declínio de Epsilon. No começo do treinamento começamos com
Epsilon=1.0, ou seja, com ações totalmente aleatórias. Depois, durante os 100K
primeiros frames, Epsilon decai linearmente para 0.02,"""
EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.02

""" Pasta onde salvaremos os logs  """
train_log_dir = "/content/drive/My Drive/DQN/Logs/"
"""Parametros de retomar treino"""
LOAD_NET = True
LOAD_REPLAY_BUFFER = True
step_index = 2209310
games_played = 8800
epsilon = 0.02#EPSILON_START
"""Bufferizando para depois inserir no log"""
log_array = []
step_epsilon_loss_array = []
games_array = []

In [0]:
"""
Código referente ao Replay Buffer. A cada Step no ambiente colocamos a 
transição no Buffer, mantendo apenas um número fixo de Steps, nesse caso 10k 
transições. Para treinamento, selecionamos aleatóriamente um lote (batch) de
transições do Replay Buffer, o que permite que quebremos a correlação entre 
passos subsequentes no ambiente.
"""
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.bool), np.array(next_states)

""" 
Agente:interage com o ambiente e salva o resultado da interação no 
experience replay buffer
"""
class Agent:
    """
    Na inicialização do Agente guardamos referências para o ambiente (env) e 
    para o experience replay buffer, registrando a observação(observation) atual
    e a recompensa total acumulada até então.
    """
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0
    """
    Faz um step no ambiente e guarda o resultado no Buffer. Com probabilidade 
    Epsilon tomamos uma ação aleatória, caso contrário utilizamos o modelo para 
    obter os Q-values para todas as possíveis ações e escolhemos a melhor.
    """
    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            epsilon_action = True
            action = env.action_space.sample()
        else:
            epsilon_action = False
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())
        """
        Passamos a ação escolhida para o ambiente e pegamos a próxima observation
        e recompensa, guardamos os dados no experience buffer e tratamos o fim-
        -de-episodio. O resultado dessa função é a recompensa total acumulada se
        chegamos ao fim-de-episodio com esse step ou None caso contrário.
        """
        new_state, reward, is_done, info = self.env.step(action)
        #log
        step_epsilon_loss_array.append(step_index)
        if epsilon_action:
            step_epsilon_loss_array.append(1)
        else:
            step_epsilon_loss_array.append(0)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward, info

"""
Função que calcula a Loss para a batch amostrada.
Argumentos: a batch como uma tupla de arrays (método sample() do experience 
buffer), nossa rede de treino e a rede alvo, periodicamente sincronizada com a 
de treino.
"""
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch
    """
    Empacota arrays individuais NumPy com dados do lote(batch) em tensores
    PyTorch e copia para a GPU
    """
    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)
    """
    Passa as observations para a primeira rede e extrai os Q-Values específicos
    para as ações tomadas (?) usando a operação de tensor gather().
    """
    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    """
    Aplica a rede alvo para as observações de próximo estado e calcula o máximo 
    Q-Value ao longo da mesma dimensão 1 de ação(?). A função max() retorna 
    ambos os valores máximos e seus índices (max e argmax), entretanto neste 
    caso, estamos apenas interesados nos valores, então pegamos a primeira 
    entrada do resultado
    """
    next_state_values = tgt_net(next_states_v).max(1)[0]
    """
    Aqui há um ponto simples porém, importante: se a transição no batch é do 
    último step do episódio, então nosso valor da ação não tem uma recompensa com
    desconto do próximo estado, visto que não há próximo estado de onde tirar 
    uma recompensa. Pode parecer pequeno mas é muito importante na prática, sem 
    isso o treino NÃO irá convergir
    """
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()
    """
    Calcula o valor da aproximação de Bellman e o erro quadrático médio (MSE)
    (loss)
    """
    expected_state_action_values = next_state_values * GAMMA + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

def write_logs(steps, games):
    with open('steps_epsilon_loss.csv', 'a', newline='') as steps_log:
        steps_writer = csv.writer(steps_log)
        for entry in steps:
            steps_writer.writerow([entry[0], entry[1], entry[2]])
        steps_log.close()
        log_array.clear()
    
    with open('games_log.csv', 'a', newline='') as games_log:
        games_writer = csv.writer(games_log)
        for entry in games:
            games_writer.writerow([entry[0], entry[1], entry[2], entry[3], entry[4]])
        games_log.close()
        games_array.clear()

In [10]:
if __name__ == "__main__":
    # Se cuda estiver disponível setamos device como cuda
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    """
    Cria ambiente (env) com todos os empacotadores aplicados, a rede neural que 
    será treinada e a rede alvo com a mesma arquitetura. No início serão 
    inicializadas com pesos diferentes, mas isso não importa visto que serão 
    sincronizadas a cada SYNC_TARGET_FRAMES, o que corresponde aproximadamente
    a um episódio de Pong (quantos frames tem um episódio de Mario?)
    """
    env = wrappers.make_env(DEFAULT_ENV_NAME)
    net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
    tgt_net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
    #Se vamos retomar o treino, carregamos a rede com os parametros salvos e 
    #sincronizamos com a target_network 
    if LOAD_NET:
        print("Reloading net: game %d, step_index %d, epsilon %.2f" % 
            (games_played, step_index, epsilon))
        net.load_state_dict(torch.load('best.dat', map_location=lambda storage, loc: storage))
        tgt_net.load_state_dict(net.state_dict())
    """
    Cria o experience replay buffer de tamanho REPLAY_SIZE e o passa para o agente.
    Epsilon é inicializado com 1.0, mas decaira a cada iteração.
    """
    #Se retomar treino, carrega o replay buffer
    if LOAD_REPLAY_BUFFER:
        print("Reloading Replay Buffer")
        with open('experienceBuffer', 'rb') as filehandler: 
            buffer = pickle.load(filehandler)
    else:
        buffer = ExperienceBuffer(REPLAY_SIZE)
    agent = Agent(env, buffer)
    """
    A última parte antes do loop de treino consiste em: criar o otimizador e 
    variavéis de tempo.Toda vez que a executarmos um numero arbitrario de jogos,
    salvamos o modelo em um arquivo.
    """
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    ts = time.time()

    while True:
        """
        No início do loop de treino, contamos o número de iterações completadas e 
        decrescemos Epsilon de acordo com o planejado. Epsilon decairá linearmente 
        durante o número de frames dados (EPSILON_DECAY_LAST_FRAME) e será mantido
        no mesmo nível que EPSILON_FINAL.
        """
        step_index += 1
        epsilon = max(EPSILON_FINAL, EPSILON_START - step_index / EPSILON_DECAY_LAST_FRAME)
        """
        Faz o agente dar um único step (usando a rede atual e o valor de epsilon).
        Essa função não retorna None apenas se este Step for o último do episódio,
        nesse caso, relatamos nosso progresso. Especificamente, calcula-se e 
        mostra tanto no console quanto no log, esses valores:
        - Contagem de episódios reproduzidos
        - Valor atual de Epsilon
        """
        reward, info = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            games_played += 1
            time_spent_game = time.time() - ts
            ts = time.time()
            print("steps[%d]: game [%d], reward [%.3f], eps [%.2f], time spent %.2f s" % (
                step_index, games_played, reward, epsilon, time_spent_game
            ))
            games_array.append([games_played, reward, epsilon, time_spent_game, info.get("x_pos")])
            # Salva a rede, replay_buffer e preenche logs a cada 'x' jogos
            if games_played % 100 == 0:
                torch.save(net.state_dict(), 'best.dat')
                with open('experienceBuffer', 'wb') as filehandler:
                    pickle.dump(buffer, filehandler)
                write_logs(log_array, games_array)
                print("Checkpoint! Game: %d, reward: %.3f, epsilon %.2f" % 
                    (games_played, reward, epsilon))
                with open('checkpoint.txt', 'w', newline='') as txt_log:
                    txt_log.write("Checkpoint! Game #%d, step_index: %d, eps: %.2f" 
                                  % (games_played, step_index, epsilon))
            if games_played > 10000:
                print("Done %d games_played!" % games_played)
                break
        """
        Checa se o Replay Buffer é grande o bastante para começar o treino. No
        começo devemos esperar REPLAY_SIZE transições. A condição seguinte 
        sincroniza os parâmetros da rede principal e da rede alvo a cada SYNC_TARGET_FRAMES.
        """
        if len(buffer) < REPLAY_START_SIZE:
            step_epsilon_loss_array.clear()
            continue
        if step_index % SYNC_TARGET_FRAMES == 0:
            tgt_net.load_state_dict(net.state_dict())
        """
        A última parte do loop de treino é simples, mas requer a maior parte 
        para executar: zera os gradientes, amostra lotes de dados do experience 
        replay buffer, calcula perca (loss) e faz o passo de otimização para 
        minimizar a perca.
        """
        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        loss_t = calc_loss(batch, net, tgt_net, device=device)
        loss_t.backward()
        optimizer.step()
        # log
        step_epsilon_loss_array.append(loss_t.item())
        log_array.append(step_epsilon_loss_array.copy())
        step_epsilon_loss_array.clear()


steps[2654071]: game [9897], reward [1692.000], eps [0.02], time spent 4.69 s
steps[2654545]: game [9898], reward [1970.000], eps [0.02], time spent 12.59 s
steps[2654574]: game [9899], reward [248.000], eps [0.02], time spent 0.77 s
steps[2654755]: game [9900], reward [1675.000], eps [0.02], time spent 4.80 s
Checkpoint! Game: 9900, reward: 1675.000, epsilon 0.02
steps[2654921]: game [9901], reward [1343.000], eps [0.02], time spent 10.73 s
steps[2655149]: game [9902], reward [1936.000], eps [0.02], time spent 6.30 s
steps[2655425]: game [9903], reward [2358.000], eps [0.02], time spent 7.91 s
steps[2655557]: game [9904], reward [1327.000], eps [0.02], time spent 3.85 s
steps[2656123]: game [9905], reward [2552.000], eps [0.02], time spent 15.53 s
steps[2656342]: game [9906], reward [1937.000], eps [0.02], time spent 5.84 s
steps[2656586]: game [9907], reward [795.000], eps [0.02], time spent 6.48 s
steps[2658591]: game [9908], reward [1915.000], eps [0.02], time spent 52.51 s
steps[2